<!--html_preserve-->
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async src="https://www.googletagmanager.com/gtag/js?id=UA-130562131-1"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-130562131-1');
</script>
<!--/html_preserve-->


In [ ]:
knitr::include_graphics("https://slcladal.github.io/images/uq1.jpg")



# Introduction{-}

This tutorial introduces power analysis using R. The entire R markdown document for this tutorial can be downloaded [here](https://slcladal.github.io/pwr.Rmd). The basis for the present tutorial is @green2016simr (which you can find [here](https://besjournals.onlinelibrary.wiley.com/doi/full/10.1111/2041-210X.12504)). @green2016simr is a highly recommendable and thorough tutorial on performing power analysis in R. Recommendable literature on this topic are, e.g. @arnold2011simulation and @johnson2015power and [this tutorial](https://www.journalofcognition.org/articles/10.5334/joc.10/). 

>
> Power analysis is a method primarily used to determine the appropriate sample size for empirical studies. 
>

Power analysis have also been used post-hoc to test if the sample size of studies was sufficient to detect meaningful effects. However, such post-hoc power calculations where the target effect size comes from the data, give misleading results (@hoenig2001abuse) and should thus be treated with extreme care.


## What determines if you find an effect?{-}

There are different factors that determine if a model finds an effect. The accuracy (i.e., the probability of finding an effect) depends on three main factors:

* the size of the effect (bigger effects are easier to detect)
* the variability of the effect (less variability makes it easier to detect an effect), and 
* the sample size (the bigger the sample size, the easier it is to detect an effect); 
  + number of subjects/participants
  + number of items/questions
  + number of observations per item within subjects/participants
  
Now, if a) we dealing with a very big effect, then we need only few participants and few items to accurately find this effect.

Or b) if we dealing with an effect that has low variability (it is observable for all subjects with the same strength), then we need only few participants and few items to accurately find this effect.

Before we conduct a study, we should figure out, what sample we need to detect a small/medium effect with medium variability so that our model is sufficient to detect this kind of effect. In order to do this, we would generate a data set that mirrors the kind of data that we expect to get (with the properties that we expect to get). We can then fit a model to this data and check if a model would be able to detect the expected effect. However, because a single model does not tell us that much (ift could simply be luck that it happened to find the effect), we run many different models on variations of the data and see how many of them find the effect. As a general rule of thumb, we want a data set that allows a model to find a medium sized effect with at least an accuracy of 80 percent [@field2007making].

In the following, we will go through how to determine what sample size we need for an example analysis.


## Preparation and session set up{-}

This tutorial is based on R. If you have not installed R or are new to it, you will find an introduction to and more information how to use R [here](https://slcladal.github.io/intror.html). For this tutorials, we need to install certain *packages* into the R *library* on your computer so that the scripts shown below are executed without errors. Before turning to the code below, please install the packages by running the code below this paragraph. If you have already installed the packages mentioned below, then you can skip ahead and ignore this section. To install the necessary packages, simply run the following code - it may take some time (between 1 and 5 minutes to install all of the libraries so you do not need to worry if it takes some time).


In [ ]:
# set options
options(stringsAsFactors = F)         # no automatic data transformation
options("scipen" = 100, "digits" = 4) # suppress math annotation
# install libraries
install.packages(c("tidyverse", "lme4", "sjPlot", "simr"))


In [ ]:
library(DT)
library(knitr)
library(flextable)


Once you have installed R and RStudio and initiated the session by executing the code shown above, you are good to go.

## Generating data{-}

In order to perform a  power analysis, we will start by loading the tidyverse package to process the data and by generating a data that we will use to determine the power of a regression model.

This simulated data set has

* 200 data points
* 2 Conditions (Control, Test)
* 10 Subjects
* 10 Items


In [ ]:
# load package
library(tidyverse)
# generate data
simdat <- data.frame(
  sub <- rep(paste0("Sub", 1:10), each = 20),
  cond <- rep(c(
    rep("Control", 10),
    rep("Test", 10))
    , 10),
  itm <- as.character(rep(1:10, 20))
) %>%
  dplyr::rename(Subject = 1,
                Condition = 2,
                Item = 3) %>%
  dplyr::mutate_if(is.character, factor)


In [ ]:
# inspect data
simdat %>%
  head(15) %>%
  kable(caption = "First 15 rows of simdat.") %>%
  kableExtra::kable_styling(bootstrap_options = c("striped", "hover", "condensed", "responsive"), 
                            full_width = F)


We add a dependent variable (AOI) which represents the dependent variable in the study. In our case, we determine that Condition has a relatively weak effect (the probability of gazing into the area of interest (AOI) is .7 in the test condition compared to .5 in the control condition). In addition, this effect is only present in half of the subjects to reflect the variability in the effect. 



In [ ]:
simdat <- simdat %>%
  dplyr::arrange(Condition) %>%
  dplyr::mutate(
  dep <- c(sample(c("AOI", "NotAOI"), 50, replace = T, prob = c(.5, .5)),
           sample(c("AOI", "NotAOI"), 50, replace = T, prob = c(.5, .5)),
           sample(c("AOI", "NotAOI"), 50, replace = T, prob = c(.5, .5)),
           sample(c("AOI", "NotAOI"), 50, replace = T, prob = c(.7, .3)))
  ) %>%
  dplyr::mutate_if(is.character, factor) %>%
  dplyr::rename(AOI = 4)


The data looks like this. 



In [ ]:
DT::datatable(simdat, rownames = FALSE, filter="none", caption = "Overview of the data set.", 
              options = list(pageLength = 10, scrollX=T))


# Post-Hoc Power Analysis

Now that we have generated some data, we will fit a model to it and perform a power analysis on the observed effects. 

>
> Post-hoc power calculations (where the target effect size comes from the data) give misleading results (@hoenig2001abuse) and should thus be treated with extreme care!
>

We will fit a first model to the data. Thus, in a first step, we load the `lme4` package to create a model, set a seed (to save the results and so that the results can be replicated), and then create an initial mixed-effects model.


In [ ]:
# load package
library(lme4)
# set seed for replicability
set.seed(12345)
# fit model
m1 <- glmer(AOI ~ (1|Subject) +(1|Item) + Condition, family="binomial", data=simdat)
# inspect results
summary(m1)


We now check the effect sizes of the predictors in the model. We can do this by displaying the results of the model using the `tab_model` function from the `sjPlot` package.



In [ ]:
# load package
library(sjPlot)
# tabulate results
sjPlot::tab_model(m1)


Now, we perform power analysis on an observed effect. This analysis tells us how likely the model is to find an observed effect given the data.

***
>
> We use a very low number of simulations (100) and we use the default z-test which is suboptimal for small samples (@bolker2009generalized). In a proper study, you shuld incerase the number of simulations (to at least 1000) and you should use a bootstrapping rather than a z-test (see @halekoh2014kenward).
>

***


In [ ]:
# load package
library(simr)
# set seed for replicability
set.seed(12345)
# perform power analysis for present model
rsim0 <- powerSim(m1, fixed("ConditionTest", "z"), nsim=100)


Now we can inspect the results. 



In [ ]:
# inspect results
rsim0


The results of the power analysis show that, given the data at hand, the model would have detected the effect of Conidition:Test with a probability of `r rsim0$x` percent. However, and as stated above, the results of such post-hoc power calculations (where the target effect size comes from the data) give misleading results (@hoenig2001abuse) and should thus be treated with extreme care!

# Power Analysis of Set Effects

While the effect of Condidition:Test is rather small (given the small number of subjects and items, the small effect size, and the variability in the effect and we would thus not be surprised that out model is not very accurate in detecting this effect), the target accuracy of finding an effect that one is interested in is commonly 80 percent (see @field2012discovering, @green2016simr). The accuracy (i.e., the probability of finding an effect) depends on three main factors:


* the size of the effect (bigger effects are easier to detect)
* the variability of the effect (less variability makes it easier to detect an effect), and 
* the sample size (the bigger the sample size, the easier it is to detect an effect); 
  + number of subjects/participants
  + number of items/questions
  + number of observations per item within subjects/participants
  

We will now check if the sample size is sufficient to detect a small effect (Cohen's d 0.2). According to @chen2010big odds ratios of 1.68, 3.47, and 6.71 are equivalent to Cohen's d = 0.2 (small), 0.5 (medium), and 0.8 (large) - the traditional scale is 0.2 for a small, 0.5 for medium sized, and 0.8 for a large or strong effect. We need to determine the odds ratios of the fixed effects and then convert them into Cohen's d values for which we have associations between traditional denominations (small, medium, and large) and effect sife values. 


In [ ]:
estimatesfixedeffects <- fixef(m1)
exp(estimatesfixedeffects)


We can see that the effect is rather small which makes it very hard to detect an effect. We will now change the size of the effect of ConditionTest to represent a truly *small* effect, i.e. on the brink of being noise but being just strong enough to be considered small. In other words, we will set the effect so that its odds ratio is exactly 1.68.  



In [ ]:
# set seed for replicability
set.seed(12345)
# perform power analysis for small effect
fixef(m1)["ConditionTest"] <- 0.519
estimatesfixedeffects <- fixef(m1)
exp(estimatesfixedeffects)


Now we can test if the sample size of the model is sufficient to find a small effect. 



In [ ]:
# set seed for replicability
set.seed(12345)
# perform power analysis for present model
rsim1 <- powerSim(m1, fixed("ConditionTest", "z"), nsim=100)


The results are shown below. 



In [ ]:
# show results
rsim1


The power analysis shows that the data is sufficient to detect a small effect for Condition:Test with `r rsim1$x` percent accuracy.

# Power Analysis of Extended Data

We will now extend the data to see what sample size is needed to get to the 80 percent accuracy threshold. We begin by increasing the number of items from 10 to 30 to see if this would lead to a sufficient sample size.


In [ ]:
# increase sample size
m2 <- extend(m1, along="Item", n=30)
# perform power simulation
rsim2 <- powerSim(m2, fixed("ConditionTest", "z"), nsim=100)


The results are shown below. 



In [ ]:
# show results
rsim2


By increasing the number of items to 30, we would now be able to detect a small effect (d=.2) with an accuracy of `r rsim2$x` percent. This means that we would have to add more items as 30 is not yet sufficient.

# Power Analysis for Ranges

We can also check the accuracy for a range of values as shown below. We begin by extending the number of Items.


In [ ]:
pc2 <- powerCurve(m2, fixed("ConditionTest", "z"), along = "Item", nsim=100)



The results are shown below. 



In [ ]:
# show results
print(pc2)


In addition, we can plot the results as follows:



In [ ]:
plot(pc2)



Instead of increasing the number of Items, we could also increase the number of Subjects. So below, we test check the accuracy for up to 30 subjects.



In [ ]:
m3 <- extend(m1, along="Subject", n=30)
# perform power calculation
pc3 <- powerCurve(m3, fixed("ConditionTest", "z"), along="Subject", nsim=100)


The results are shown below. 



In [ ]:
# print results
print(pc3)


Again, we can also visualize the results.



In [ ]:
# visualize results
plot(pc3) 


The results show that we breach the 80 percent threshold with 30 subjects.

Finally, it may be an option to increase the number of data points within subjects and items (while the number of items and subjects remain constant). 


In [ ]:
m4 <- extend(m1, within="Item+Subject", n=15)
# perform power calculation
pc4 <- powerCurve(m4, fixed("ConditionTest", "z"), within="Item+Subject", breaks=c(5, 10, 15), nsim=100)


The results are shown below. 



In [ ]:
# show results
print(pc4)


In [ ]:
# show results
plot(pc4)


The results show that we would have a sufficient data set if we had 14 observations per Subject in each Item because with 10 observations, the accuracy breaches the 80 percent level.

# Citation & Session Info {-}

Schweinberger, Martin. `r format(Sys.time(), '%Y')`. *Power Analysis in R*. Brisbane: The University of Queensland. url: https://slcladal.github.io/clust.html (Version `r format(Sys.time(), '%Y.%m.%d')`).


In [ ]:
@manual{schweinberger`r format(Sys.time(), '%Y')`pwr,
  author = {Schweinberger, Martin},
  title = {Power Analysis in R},
  note = {https://slcladal.github.io/pwr.html},
  year = {`r format(Sys.time(), '%Y')`},
  organization = "The University of Queensland, Australia. School of Languages and Cultures},
  address = {Brisbane},
  edition = {`r format(Sys.time(), '%Y.%m.%d')`}
}


In [ ]:
sessionInfo()



***

[Back to top](#introduction)

[Back to HOME](https://slcladal.github.io/index.html)

***

# References {-}
